In [5]:
from typing import Tuple
import numpy as np
import torch
import torch.utils.data as data
from torch.autograd import Variable
from net import HGCTNet

In [6]:
# model = HGCTNet(num_classes=2, num_hid=16, depth=2, num_heads=2, hga=True, trans=True)
# sigs = Variable(torch.randn(5, 3, 12000))
# feas = Variable(torch.randn(5, 53))
# estbp = model(sigs, feas)

In [7]:
def create_dataset(dataset: np.ndarray, window_size: int) -> Tuple[torch.Tensor]:
    """
    Transform a time series into a prediction dataset.
    """
    x, y = [], []
    for i in range(window_size, len(dataset)):
        feature = dataset[i - window_size: i].reshape(window_size, -1)
        feature = feature / feature.sum(axis=0, keepdims=True)
        target = (2 * (dataset[i, :, 3] > dataset[i - 1, :, 0]) - 1) * (dataset[i, :, 3] != dataset[i - 1, :, 0])
        x.append(feature)
        y.append(target)
    x, y = np.array(x, dtype=np.float32), np.array(y, dtype=np.float32)
    return torch.Tensor(x), torch.Tensor(y)

In [12]:
test_feature = torch.load('test_feature.pt')
test_signal = torch.load('test_signal.pt')

In [ ]:
test_feature.shape

In [ ]:
test_signal.shape

In [ ]:
train_x, train_y = create_dataset(row_data[: 1440 * 7 * 4], 1440)
valid_x, valid_y = create_dataset(row_data[1440 * (7 * 4 - 1): 1440 * (7 * 4 + 1)], 1440)
train_loader = data.DataLoader(data.TensorDataset(train_x, train_y), shuffle=True, batch_size=512, num_workers=12)
valid_loader = data.DataLoader(data.TensorDataset(valid_x, valid_y), shuffle=True, batch_size=512, num_workers=12)